In [8]:
import argparse
import cv2
import dlib
import numpy as np
import tensorflow as tf
from imutils import video


import os
from glob import glob
import scipy.io as sio
from skimage.io import imread, imsave
from skimage.transform import rescale, resize
from time import time
import ast

from api import PRN

from utils.estimate_pose import estimate_pose
from utils.rotate_vertices import frontalize
from utils.render_app import get_visibility, get_uv_mask, get_depth_image
from utils.write import write_obj_with_colors, write_obj_with_texture
from utils.cv_plot import plot_kpt, plot_vertices, plot_pose_box

import sys
#sys.path.append('/Users/stevenye/cv/face3d_old')
import face3d
from face3d import mesh
from face3d import mesh_numpy
from face3d.morphable_model import MorphabelModel

CROP_SIZE = 128
DOWNSAMPLE_RATIO = 1

def resize(image):
    """Crop and resize image for pix2pix."""
    height, width, _ = image.shape
    if height != width:
        # crop to correct ratio
        size = min(height, width)
        oh = (height - size) // 2
        ow = (width - size) // 2
        cropped_image = image[oh:(oh + size), ow:(ow + size)]
        image_resize = cv2.resize(cropped_image, (CROP_SIZE, CROP_SIZE))
        return image_resize

def do_run():

    # OpenCV
    #cap = cv2.VideoCapture(args.video_source)
    cap = cv2.VideoCapture('b.mov')
    fps = video.FPS().start()

    # ---- init PRN
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # GPU number, -1 for CPU
    prn = PRN(is_dlib = True)

    #while True:
    while cap.isOpened():
        ret, frame = cap.read()

        # resize image and detect face
        frame_resize = cv2.resize(frame, None, fx=1 / DOWNSAMPLE_RATIO, fy=1 / DOWNSAMPLE_RATIO)

        # read image
        image = frame_resize
        image = resize(image)

        [h, w, c] = image.shape
        if c>3:
            image = image[:,:,:3]

        # the core: regress position map
        if True:
            max_size = max(image.shape[0], image.shape[1])
            if max_size> 1000:
                image = rescale(image, 1000./max_size)
                image = (image*255).astype(np.uint8)
            st = time()
            pos = prn.process(image) # use dlib to detect face
            print('process',time()-st)
        else:
            if image.shape[0] == image.shape[1]:
                image = resize(image, (256,256))
                pos = prn.net_forward(image/255.) # input image has been cropped to 256x256
            else:
                box = np.array([0, image.shape[1]-1, 0, image.shape[0]-1]) # cropped with bounding box
                pos = prn.process(image, box)
        
        image = image/255.
        if pos is None:
            cv2.imshow('a',frame_resize)
            fps.update()
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        if True:
            # 3D vertices
            vertices = prn.get_vertices(pos)
            if False:
                save_vertices = frontalize(vertices)
            else:
                save_vertices = vertices.copy()
            save_vertices[:,1] = h - 1 - save_vertices[:,1]

        rendering_cc = mesh.render.render_grid(save_vertices, prn.triangles, 900, 900)
        a = np.transpose(rendering_cc,axes=[1,0,2])
        dim = rendering_cc.shape[0]

        i_t = np.ones([dim,dim,3],dtype= np.float32)
        for i in range(dim):
            i_t[i] = a[dim-1-i]
        i_t = i_t/255

        cv2.imshow('sparse alignment', i_t)
        
        fps.update()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    fps.stop()
    print('[INFO] elapsed time (total): {:.2f}'.format(fps.elapsed()))
    print('[INFO] approx. FPS: {:.2f}'.format(fps.fps()))

    cap.release()
    cv2.destroyAllWindows()



do_run()


ValueError: Variable resfcn256/Conv/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/usr/local/lib/python3.5/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 277, in variable
    aggregation=aggregation)
  File "/usr/local/lib/python3.5/site-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 182, in func_with_args
    return func(*args, **current_args)
  File "/usr/local/lib/python3.5/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 350, in model_variable
    aggregation=aggregation)
